# Mean Variance Optimization with CVXOPT and CVXPY

In [3]:
import sys
sys.path.insert(0,'C:\\code\\python_for_the_financial_economist\\')

# import relevant packages
import numpy as np
import pandas as pd
from scipy import stats, optimize
import matplotlib.pyplot as plt

# packages for convex optimization
import cvxpy as cp
import cvxopt

# own functions from codelib
from codelib.statistics import moments as mom
from codelib.portfolio_optimization.mean_variance import portfolio_mean, portfolio_std, portfolio_variance, minimum_variance_portfolio

from codelib.visualization.layout import DefaultStyle
DefaultStyle();

Consider the portfolio optimization problem 


where the expected return vector is given by 

$$
\boldsymbol{\mu} = \begin{pmatrix} 0.032 \\ 0.0322 \\ 0.084 \\ 0.082 \end{pmatrix},
$$

the vector of volatilites is given by 

$$
\mathbf{v} = \begin{pmatrix} 0.05 \\ 0.05 \\ 0.22 \\ 0.22 \end{pmatrix},
$$

and the correlation matrix is given by 

$$
\mathbf{C} = \begin{pmatrix} 1.0 & 0.85 & 0.5 & 0.45 \\
                      0.85 & 1.0 & 0.5 & 0.45 \\
                      0.5 & 0.5 & 1.0 & 0.9 \\
                      0.45 & 0.45 & 0.9 & 1.0 \end{pmatrix}
$$

In [4]:
corr_mat = np.array([[1.0, 0.85, 0.5, 0.45],
                     [0.85, 1.0, 0.5, 0.45],
                     [0.5, 0.5, 1.0, 0.9],
                     [0.45, 0.45, 0.9, 1.0]])

vols = np.array([5.0, 5.0, 22.0, 22.0]) / 100.0
mu = np.array([3.2, 3.22, 8.4, 8.2]) / 100.0

cov_mat = mom.corr_to_cov_matrix(corr_mat, vols)

## Problem 1

Set up the minimization problem that enable you to find the minimum variance portfolio using both `cvxopt` and `cvxpy`. Apply a budget constraint requiring the portfolio weights to sum to one. Compare with the analytical solution. 

### Solution

We have the minimization problem 

$$
\mathbf{w}^{Min-Var} = \arg \min \mathbf{w}^\top \boldsymbol{\Sigma} \mathbf{w} \; \; \text{st. } \mathbf{1}^\top \mathbf{w} = 1
$$

The analytical solution is given by 

$$
\mathbf{w}^{Min-Var} = \frac{\boldsymbol{\Sigma}^{-1} \mathbf{1}}{\mathbf{1}^\top \boldsymbol{\Sigma}^{-1} \mathbf{1}}
$$

In [11]:
# number of assets
num_assets = len(mu)

In [10]:
"""
Solving with CVXOPT
"""

P = cvxopt.matrix(cov_mat) #<- we need to minimize
q = cvxopt.matrix(np.zeros(num_assets)) 
A = cvxopt.matrix([[1.0, 1.0, 1.0, 1.0]]).T
b = cvxopt.matrix([1.0])

# solve problem 
sol = cvxopt.solvers.qp(P, q, A=A, b=b)

# solution 
np.array(sol['x']) # <- gives the same solution as the analytical formula

array([[ 0.53892691],
       [ 0.53892691],
       [-0.1001686 ],
       [ 0.02231479]])

In [5]:
"""
Solving with CVXPY
"""

# define variables
P = cov_mat
A = np.ones(num_assets)
b = 1.0


# define problem 
x = cp.Variable(num_assets)
prob = cp.Problem(cp.Minimize(cp.quad_form(x, P)),
                  constraints=[cp.sum(x) == 1]) # cp.sum(x) == 1 or  A @ x == b

# solve problem
prob.solve()

# solution 
x.value # <- gives the same solution as the analytical formula

array([ 0.53892691,  0.53892691, -0.1001686 ,  0.02231479])

In [9]:
"""
Solving analytically
"""

minimum_variance_portfolio(cov_mat)

array([ 0.53892691,  0.53892691, -0.1001686 ,  0.02231479])

## Problem 2

Minimize the portfolio variance subject to the constraints that using the `CVXPY` or `CVXOPT` package. 

* The expected return should be above 6%
* No shorting (all weights should be equal or greater than 0)
* The portfolio weights need to sum to one (budget constraint)

### Solution

In [27]:
"""
Solving with CVXOPT
"""

# objectiv function
P = cvxopt.matrix(cov_mat) #<- we need to minimize
q = cvxopt.matrix(np.zeros(num_assets)) 
# equality constraint
A = cvxopt.matrix([[1.0, 1.0, 1.0, 1.0]]).T
b = cvxopt.matrix([1.0])
# inequality constraints
G = cvxopt.matrix(np.vstack((-np.eye(num_assets),
                             -mu)))
h = cvxopt.matrix(np.r_[np.zeros(num_assets), -0.06])

# solve problem 
sol = cvxopt.solvers.qp(P, q, A=A, b=b, G=G, h=h)

# solution 
np.array(sol['x']) # <- gives the same solution as the analytical formula


     pcost       dcost       gap    pres   dres
 0:  6.8954e-03 -9.9866e-01  6e+00  2e+00  3e+00
 1:  7.8252e-03 -6.4361e-01  7e-01  3e-02  3e-02
 2:  8.5319e-03 -1.8427e-02  3e-02  1e-03  1e-03
 3:  8.5500e-03  7.6858e-03  9e-04  3e-05  4e-05
 4:  8.4181e-03  8.3773e-03  4e-05  1e-06  1e-06
 5:  8.4086e-03  8.4056e-03  3e-06  1e-16  1e-15
 6:  8.4082e-03  8.4081e-03  1e-07  1e-16  2e-14
 7:  8.4082e-03  8.4082e-03  1e-09  6e-17  4e-14
Optimal solution found.


array([[0.10446003],
       [0.35062141],
       [0.34197392],
       [0.20294464]])

In [14]:
"""
Solving with CVXPY
"""

# define variables
P = cov_mat
A = np.ones(num_assets)
b = 1.0


# define problem 
x = cp.Variable(num_assets)
prob = cp.Problem(cp.Minimize(cp.quad_form(x, P)),
                  constraints=[cp.sum(x) == 1, 
                               x @ mu >= 0.06,
                               x >= 0])
# solve problem
prob.solve()

# solution 
x.value # <- gives the same solution as the analytical formula

array([0.10444791, 0.35063358, 0.34197387, 0.20294464])